# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# More Dependencies
import numpy as np
import json


In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,28.09,100,75,3.60,TO,1675236006
1,1,khingansk,49.1167,131.1833,-20.55,94,99,4.92,RU,1675236007
2,2,saskylakh,71.9167,114.0833,-25.76,96,100,9.09,RU,1675236008
3,3,new norfolk,-42.7826,147.0587,18.29,34,4,1.79,AU,1675236008
4,4,westport,41.1415,-73.3579,-0.25,57,100,4.63,US,1675236011


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "Country",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols=['City'],
    size = 'Humidity'
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# cities with humidity less than or equal to 50
# cities with max temps less than or equal to 30
# cities with cloudiness >= 20
ideal_city_df = city_data_df[(city_data_df["Max Temp"] <=30) & (city_data_df["Humidity"] <= 50) & (city_data_df["Cloudiness"] >=20.0)]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,hobart,-42.8794,147.3294,18.48,43,20,7.72,AU,1675235868
88,88,lalmohan,22.3384,90.7419,26.87,41,67,5.74,BD,1675236057
91,91,kurduvadi,18.0833,75.4333,28.61,29,65,6.24,IN,1675236059
98,98,bubaque,11.2833,-15.8333,24.99,47,54,6.08,GW,1675236063
113,113,mopti,14.6667,-4.0000,17.14,11,100,4.01,ML,1675236072


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()





,City,Country,Lat,Lng,Humidity,Hotel Name
61,hobart,AU,-42.8794,147.3294,43,
88,lalmohan,BD,22.3384,90.7419,41,
91,kurduvadi,IN,18.0833,75.4333,29,
98,bubaque,GW,11.2833,-15.8333,47,
113,mopti,ML,14.6667,-4.0000,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hobart - nearest hotel: Vibe Hotel Hobart
lalmohan - nearest hotel: No hotel found
kurduvadi - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
mopti - nearest hotel: No hotel found
baiyin - nearest hotel: 红鹭宾馆
ferkessedougou - nearest hotel: Ferké Hôtel Iroko
adrar - nearest hotel: No hotel found
wamba - nearest hotel: No hotel found
nouakchott - nearest hotel: فندق أدرار
taoudenni - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
lower hutt - nearest hotel: Angus Inn
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
mongo - nearest hotel: No hotel found
warrnambool - nearest hotel: Central Court
parakou - nearest hotel: Hôtel "Les Routier"
ranavav - nearest hotel: Hotel Shiv
changping - nearest hotel: 速8酒店
talaja - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
hanzhong - nearest hotel: 汉庭酒店
oussouye - nearest hotel: Campement Alulum
mangrol - nearest hotel: No hotel found
hasaki - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
61,hobart,AU,-42.8794,147.3294,43,Vibe Hotel Hobart
88,lalmohan,BD,22.3384,90.7419,41,No hotel found
91,kurduvadi,IN,18.0833,75.4333,29,No hotel found
98,bubaque,GW,11.2833,-15.8333,47,Chez Julio
113,mopti,ML,14.6667,-4.0000,11,No hotel found
143,baiyin,CN,36.5583,104.2081,23,红鹭宾馆
158,ferkessedougou,CI,9.6000,-5.2000,16,Ferké Hôtel Iroko
161,adrar,MR,20.5022,-10.0711,16,No hotel found
181,wamba,NG,8.9333,8.6000,17,No hotel found
192,nouakchott,MR,18.0858,-15.9785,40,فندق أدرار


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Hotel Name"],
    geo = True,
    tiles = "OSM",
    color = 'Country',
    frame_width = 700,
    frame_height = 400
)


# Display the map
map_plot_2
    
    

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)